In [2]:
"""
Find a few feasible real points x* in semialgebraic set S(g,h), where
 g = {g0,g1,...,gm} are inequalities with g0=1 and h = {h1,...,hm} are equalities.

Adding sphere inequalities method:
1) Let a0,...,an in R^n such that a1-a0,...,an-a0 are linear independent
2) Solve L = min {|x|^2: x in S(g,h)} by SDP relaxation
         Lk = min Ly(theta^k(|x|^2 + eps theta^2))
              s.t. y in R^(2(k+2))
                   M(k+2-u_j)(gj y)>=0
                   M(k+2-w_j)(hj y)=0
                   Ly(theta^k)=1
where u_j=ceil(deg(g_j)/2) and w_j=ceil(deg(h_j)/2). Set g_m = Lk - |x|^2.
3) Solve xit = min{|x-at|^2: x in S(g cup {xij-|x-aj|^2: j=0,...,t-1},h)},
t=0,...,n by numerical scheme of Lasserre's hierarchies
         omegak_t = min Ly(|x-at|^2)
                  s.t. y in R^(2(k+2))
                       M(k+2-u_j)(gj y)>=0
                       M(k+1)((omegaj-|x-aj|^2) y)>=0, j=0,...,t-1
                       M(k+2-w_j)(hj y)=0              
                       Ly(1)=1
for t=0,...,n.
4) Check rank condition of each problem of optimal value omegat t in
{0,...,n} to extract supp(mu) if solution y has representing atomic
measure mu. In this case, x* in supp(mu).
5) Solve non-singular linear system Ax*=b where A=(a1-a0 ... an-a0) and
b= (-(|aj|^2 - |a0|^2 + omegaj - omega0)/2)_{j=1,...,n}.
"""



using DynamicPolynomials

using JuMP

using MosekTools

using CPUTime

using LinearAlgebra

using RowEchelon


while 1>0

start = time()





# define polnomial variables
@polyvar x1 x2 x3 x4
x=[x1;x2;x3;x4]; n=length(x)

#inequalities polynomial
g = [x1^2+x2^2+x3^2+x4^2-2*x1*x3-2*x2*x4-1]; m=length(g) 

#inequalities polynomial
h = [x1^2+x2^2-1;x3^2+x4^2-1]; l=length(h)

# index of relaxation
k = 1








# small parameter for L
eps = 1e-2

# for rank of moment matrix
TOL=1e-2

# parameter for pivot of rref
tau=1e-3


# quadraic polynomial
theta=1+x'*x



# Define centers and square of radius
a0=zeros(Float64,(n, 1)); a = Matrix{Float64}(I, n, n)



# Degree of inequalities polynomials
dg = []
for i = 1:m
    dg = [dg; ceil(Int,degree(leadingmonomial(g[i]))/2)]
end


# Degree of inequalities polynomials
dh = []
for j = 1:l
    dh = [dh; ceil(Int,degree(leadingmonomial(h[j]))/2)]
end

dmax=maximum([dg;dh;1])




println("Determine L0:")
 
println("---------------------------------------------------------------------------------------")




# Define vetor of monomials
v0= monomials(x, 0)
for j in 1:k+2
    v0= [v0;monomials(x, j)]
end

w0= monomials(x, 0)
for j in 1:2*(k+2)
    w0= [w0;monomials(x, j)]
end

length_v_max=length(v0)

length_v=[];
for i in 1:m
    length_v=[length_v; binomial(2+k-dg[i]+n,n)]
end

length_w=[];
for j in 1:l  
    length_w=[length_w;binomial(2*(2+k-dh[j])+n,n)]
end
# Define sum of square cone
model = Model(with_optimizer(Mosek.Optimizer, QUIET=true))

# Weighted SOS matrix
@variable(model, G0[1:length_v_max, 1:length_v_max],PSD)

# Weighted SOS decomposition
wSOS=v0'*G0*v0

for i in 1:m
    
    G = @variable(model, [1:length_v[i], 1:length_v[i]],PSD)
    
    wSOS = wSOS + g[i]*v0[1:length_v[i]]'*G*v0[1:length_v[i]]
    
end


for j in 1:l
    
    q = @variable(model, [1:length_w[j]])
    
    wSOS = wSOS + h[j]*w0[1:length_w[j]]'*q
    
end

@variable(model, lambda)

@constraint(model, coefficients(theta^k*(x'*x - lambda + eps*theta^2) - wSOS) .== 0)

@objective(model, Max, lambda)

optimize!(model)

L0 = value(lambda)



println("termination status = ", termination_status(model))

println("L0 = ", L0)

println("=======================================================================================")

if termination_status(model) != MOI.SLOW_PROGRESS && termination_status(model)!= MOI.OPTIMAL
    println("Increase order k!")
    break
end










# Define omegat, t=0,...,n
omega0 = 0; omega = zeros(n)






println("Determine omega",0,":")

println("---------------------------------------------------------------------------------------")

# Define vetor of monomials
v0= monomials(x, 0)
for j in 1:k+dmax
    v0= [v0;monomials(x, j)]
end

w0= monomials(x, 0)
for j in 1:2*(k+dmax)
    w0= [w0;monomials(x, j)]
end

length_v_max=length(v0)

length_v=[];
for i in 1:m
    length_v=[length_v; binomial(dmax+k-dg[i]+n,n)]
end

length_w=[];
for j in 1:l  
    length_w=[length_w;binomial(2*(dmax+k-dh[j])+n,n)]
end

length_v_sphere=binomial(dmax+k-1+n,n)


# Define sum of square cone
model = Model(with_optimizer(Mosek.Optimizer, QUIET=true))


# Weighted SOS matrix
G0=@variable(model, [1:length_v_max, 1:length_v_max])

consG0=@constraint(model, G0 in PSDCone())

@variable(model, lambda)

# Weighted SOS decomposition
wSOS=v0'*G0*v0

for i in 1:m
    
    G = @variable(model, [1:length_v[i], 1:length_v[i]],PSD)
    
    wSOS = wSOS + g[i]*v0[1:length_v[i]]'*G*v0[1:length_v[i]]
    
end




for j in 1:l
    
    q = @variable(model, [1:length_w[j]])
    
    wSOS = wSOS + h[j]*w0[1:length_w[j]]'*q
    
end

G = @variable(model, [1:length_v_sphere, 1:length_v_sphere],PSD)

wSOS=wSOS+(L0-x'*x)*v0[1:length_v_sphere]'*G*v0[1:length_v_sphere]

@constraint(model, coefficients([x-a0]'*[x-a0]-lambda-wSOS).== 0)

@objective(model, Max, lambda)

optimize!(model)

omega0 = value(lambda)

M=dual(consG0)
rM=rank(M, TOL)
dimsubM=binomial(k+n,n)
subM=M[1:dimsubM,1:dimsubM]
rsubM=rank(subM, TOL)

println("termination status = ", termination_status(model))

println("omega",0," = ", value(lambda))

println("rank of moment matrix = ", rM)

println("rank of moment submatrix = ", rsubM)

if termination_status(model) != MOI.SLOW_PROGRESS && termination_status(model)!= MOI.OPTIMAL
    println("Increase order k!")
    break
end

# extraction of Henrion and Lasserre
if rM==rsubM # check the flat-extension
    F = eigen(value.(G0))
    V = F.vectors
    r=rM
    Ix=sortperm(F.values)

    V=V[:,Ix[1:r]]
    V=Matrix(V')
    V= rref_with_pivots!(V,tau);
    U=V[1]

    U=Matrix(U')
    # Figure out multiplying matrices using YALMIP code
    w=v0[V[2]];
    N=zeros(length(V[2]),r,n)
    for i in 1:n
        xw=x[i]*w
        kk=indexin(xw,v0)
        N[:,:,i]=U[kk,:]
    end



    # Create random convex combination
    rands = rand(n,1);rands = rands/sum(rands);
    M = zeros(length(V[2]),r);
    for i in 1:n
        M=M+rands[i]*N[:,:,i];
    end

    F= schur(M);
    L=F.Z
    # Extract solution
    for i in 1:r
        solution=[]
        for j = 1:n
            solution=[solution;L[:,i]'*N[:,:,j]*L[:,i]];
        end

        println("solution = ",solution)
        #check the feasibility of solution 
        for i in 1:m
            println("check inequality ",i," = ",polynomial(g[i])(x => solution))         
        end

        for i in 1:l
            println("check equality ",i," = ",polynomial(h[i])(x => solution))
        end
        elapsed = time() - start
        println("elapsed time = ",elapsed)
        println("---------------------------------------------------------------------------------------")
    end
    break

 end


 println("=======================================================================================")
    





length_v_sphere=binomial(2*(dmax+k-1)+n, n)

for j in 1:n

    println("Determine omega_k^",j,":")

    println("---------------------------------------------------------------------------------------")

    # Define sum of square cone
    model = Model(with_optimizer(Mosek.Optimizer, QUIET=true))
    
    # Weighted SOS matrix
    G0=@variable(model, [1:length_v_max, 1:length_v_max])

    consG0=@constraint(model, G0 in PSDCone())

    @variable(model, lambda)

    # Weighted SOS decomposition
    wSOS=v0'*G0*v0

    for i in 1:m

        G = @variable(model, [1:length_v[i], 1:length_v[i]],PSD)

        wSOS = wSOS + g[i]*v0[1:length_v[i]]'*G*v0[1:length_v[i]]

    end

    for j in 1:l
        q = @variable(model, [1:length_w[j]])

        wSOS = wSOS + h[j]*w0[1:length_w[j]]'*q

    end

    q = @variable(model, [1:length_v_sphere])

    wSOS = wSOS + (omega0-[x-a0]'*[x-a0])*w0[1:length_v_sphere]'*q


    #G = @variable(model, [1:length_v_sphere, 1:length_v_sphere],PSD)

    #wSOS = wSOS + (L0-x'*x)*v0[1:length_v_sphere]'*G*v0[1:length_v_sphere]
    
    if j>=2
        for i in 1:j-1

            Q= @variable(model, [1:length_v_sphere])

            wSOS = wSOS + (omega[i]-[x-a[:,i]]'*[x-a[:,i]])*w0[1:length_v_sphere]'*Q

        end 
    end
    
    @constraint(model, coefficients([x-a[:,j]]'*[x-a[:,j]] - lambda - wSOS) .== 0)

    @objective(model, Max, lambda)

    optimize!(model)
    
    omega[j] = value(lambda)


    M=dual(consG0)
    rM=rank(M, TOL)
    dimsubM=binomial(k+n,n)
    subM=M[1:dimsubM,1:dimsubM]
    rsubM=rank(subM, TOL)


    println("termination status = ", termination_status(model))
    
    
    println("omega",j," = ", value(lambda))

    println("rank of moment matrix = ", rM)

    println("rank of moment submatrix = ", rsubM)

    if termination_status(model) != MOI.SLOW_PROGRESS && termination_status(model)!= MOI.OPTIMAL
        println("Increase order k!")
        break
    end
           


    if rM==rsubM
        F = eigen(value.(G0))
        V = F.vectors
        r=rM
        Ix=sortperm(F.values)
        
        V=V[:,Ix[1:r]]
        V=Matrix(V')
        V= rref_with_pivots!(V,tau);
        U=V[1]

        U=Matrix(U')
        # Figure out multiplying matrices using YALMIP code
        w=v0[V[2]];
        N=zeros(length(V[2]),r,n)
        for i in 1:n
            xw=x[i]*w
            kk=indexin(xw,v0)
            N[:,:,i]=U[kk,:]
        end



        # Create random convex combination
        rands = rand(n,1);rands = rands/sum(rands);
        M = zeros(length(V[2]),r);
        for i in 1:n
            M=M+rands[i]*N[:,:,i];
        end

        F= schur(M);
        L=F.Z
        # Extract solution
        for i in 1:r
            solution=[]
            for j in 1:n
                solution=[solution;L[:,i]'*N[:,:,j]*L[:,i]];
            end

            println("solution = ",solution)

            for i in 1:m
                println("check inequality ",i," = ",polynomial(g[i])(x => solution))         
            end

            for i in 1:l
                println("check equality ",i," = ",polynomial(h[i])(x => solution))
            end
            elapsed = time() - start
            println("elapsed time = ",elapsed)
            println("---------------------------------------------------------------------------------------")
        end
        break

     end

    
    
     println("=======================================================================================")
    
end
        






println("Compute a solution from sphere equations:")
println("---------------------------------------------------------------------------------------")

# solve linear programming

A=-a.+a0
b=[]
for j=1:n
    b=[b;omega[j]-omega0-norm(a[:,j])^2+norm(a0)^2]
end
b=.5*b

solution=inv(A)*b
   
println("solution = ",solution)

for i in 1:m
    println("check inequality ",i," = ",polynomial(g[i])(x => solution))         
end
    
for i in 1:l
    println("check equality ",i," = ",polynomial(h[i])(x => solution))
end
elapsed = time() - start
println("elapsed time = ",elapsed)
break
end

Determine L0:
---------------------------------------------------------------------------------------
termination status = OPTIMAL
L0 = 2.0899999976524084
Determine omega0:
---------------------------------------------------------------------------------------
termination status = OPTIMAL
omega0 = 1.9999999999405915
rank of moment matrix = 13
rank of moment submatrix = 5
Determine omega_k^1:
---------------------------------------------------------------------------------------
termination status = OPTIMAL
omega1 = 0.9999999980844148
rank of moment matrix = 5
rank of moment submatrix = 3
Determine omega_k^2:
---------------------------------------------------------------------------------------
termination status = SLOW_PROGRESS
omega2 = 2.999359319243128
rank of moment matrix = 5
rank of moment submatrix = 3
Determine omega_k^3:
---------------------------------------------------------------------------------------
termination status = SLOW_PROGRESS
omega3 = 2.0011528858973464
rank of